# Projeto 1

Nome: Enzo Cardoso de Almeida Santos Neto

In [1]:
#Carregando bibliotecas
import math
import os.path
import pandas as pd
import json
from random import shuffle
import re 
import matplotlib.pyplot as plt
#Caso de erro, instalar no prompt - pip install emoji
import emoji
from emoji  import UNICODE_EMOJI

### Base de Treinamento

In [2]:
#Excel para o treinamento do código
base_total = pd.read_excel("Valorant.xlsx", sheet_name="Treinamento")

#Série de todos os dados + sua tabela absoluta
serie_tudo = pd.Series(base_total.Treinamento)
tabela_tudo_absoluta = serie_tudo.value_counts()

### Etapa 3 - Removendo sinais e separação de emojis 

In [3]:
#Função para limpar os sinais básicos e separa emojis.
def remocao_de_sinais(texto):
    import string
    pontuacao = '[!-.:?;]'
    padrao = re.compile(pontuacao)
    text_subbed = re.sub(padrao, ' ', texto)
    letras=[]
    for letra in text_subbed:  
        letras.append(letra)
    i=1
    espaço=" "
    frase_separando_emoji=""
    while i<= (len(letras)-1):        
        if letras[i] in UNICODE_EMOJI:
            frase_separando_emoji+=(espaço)
            frase_separando_emoji+=letras[i-1]
            frase_separando_emoji+=(espaço)
            frase_separando_emoji+=(letras[i])
            letras.remove(letras[i])
            i+=1
        else:
            frase_separando_emoji+=(letras[i-1])
            i+=1
    return frase_separando_emoji

In [4]:
#limpando os dados de treinamento 
tweets_relevantes = open("relevante.txt", "r", encoding="utf8")
relevante_valorant = remocao_de_sinais((tweets_relevantes.read()).lower())

tweets_irrelevantes = open("irrelevante.txt", "r", encoding="utf8")
irrelevante_valorant = remocao_de_sinais((tweets_irrelevantes.read()).lower())

In [5]:
#tabelas relativas e absolutas
relevante_relativa = (pd.Series(relevante_valorant.split())).value_counts(True)
relevante_absoluta = (pd.Series(relevante_valorant.split())).value_counts()

irrelevante_relativa = (pd.Series(irrelevante_valorant.split())).value_counts(True)
irrelevante_absoluta = (pd.Series(irrelevante_valorant.split())).value_counts()

In [6]:
#probabilidades
probabilidade_relevante = relevante_absoluta.sum()/tabela_tudo_absoluta.sum()
probabilidade_irrelevante = irrelevante_absoluta.sum()/tabela_tudo_absoluta.sum()

### Etapa 4 - Performance do classificador

In [7]:
#definindo a função Laplace
def laplace(palavra, tabela1, tabela2):
    if palavra in tabela1:
        x = tabela1[palavra]
    else:
        x = 0
        
    if palavra in tabela2:
        y = tabela2[palavra]
    else:
        y = 0 
    
    prob_relevante = (x + 1)/ (tabela1.sum() + len(set(tabela1+tabela2)))
    prob_irrelevante = (y + 1)/ (tabela2.sum() + len(set(tabela1+tabela2)))
    return [prob_relevante, prob_irrelevante]

In [8]:
#lendo o excel com os tweets de teste.
base_teste = pd.read_excel("Valorant.xlsx",sheet_name="Teste")
series_tweets = pd.Series(list(base_teste.Teste))

In [9]:
#limpando os tweets do teste
tweets_prontos = []
for tweet in series_tweets:
    tweets_prontos.append(remocao_de_sinais(tweet.lower()))


#classificador
classificacao = {}
lista_classificada = []
for frase in tweets_prontos:
    palavras_da_frase = list(frase.split())
    relevante = 1
    irrelevante = 1
    
    for palavra in palavras_da_frase:
        
        # caso esteja nas duas categorias
        if palavra in relevante_absoluta and \
        palavra in irrelevante_absoluta:
            relevante *= relevante_relativa[palavra] #20
            irrelevante *= irrelevante_relativa[palavra] #21
        
        #caso esteja somente em relevante
        elif palavra in relevante_absoluta and \
        palavra not in irrelevante_absoluta:
            relevante *= relevante_relativa[palavra] #26
            irrelevante *= laplace(palavra,relevante_absoluta,irrelevante_absoluta)[1]
        
        #caso esteja somente em irrelevante
        elif palavra in irrelevante_absoluta and \
        palavra not in relevante_absoluta:
            relevante *= laplace(palavra,relevante_absoluta,irrelevante_absoluta)[0]
            irrelevante *= irrelevante_relativa[palavra] #33
        
        #caso nao esteja em nenhuma categoria
        else:
            relevante *= laplace(palavra,relevante_absoluta,irrelevante_absoluta)[0]
            irrelevante *= laplace(palavra,relevante_absoluta,irrelevante_absoluta)[1]
    
    #probabilidade da frase(tweet) ser relevante e irrelevante
    prob_relevante = probabilidade_relevante*relevante
    prob_irrelevante = probabilidade_irrelevante*irrelevante
    
    #comparacao das probabilidades
    if prob_relevante > prob_irrelevante:
        classificacao[frase] = 0
        lista_classificada.append(0)
    else:
        classificacao[frase] = 1
        lista_classificada.append(1)
        
classificados = pd.Series(classificacao)

In [10]:
#comparando o teste com a classificação 
for tweet in classificacao:
    dic={'Tweet': tweets_prontos, 'Resultado Teste': lista_classificada, 'Classificação': base_teste["Classe"]}

In [11]:
tabela_classificacao = pd.DataFrame(dic, columns=['Tweet','Resultado Teste','Classificação'])

In [12]:
acertos = 0
errados = 0

#varrendo as respostas
for classificado , certo in zip(lista_classificada, base_teste["Classe"]):
    if classificado == certo:
        acertos += 1
    else:
        errados += 1
        
porcentagem_acertos = (acertos / 200) *100
porcentagem_errados = (errados / 200) *100


print("Porcentagem de acertos = {}%".format(porcentagem_acertos))
print("Porcentagem de erros = {}%".format(porcentagem_errados))

Porcentagem de acertos = 56.00000000000001%
Porcentagem de erros = 44.0%


In [13]:
verdadeiros_positivo = 0
falsos_positivos = 0
verdadeiros_negativos = 0
falsos_negativos = 0

# relevante = 1 e irrelevante = 0

for resultado_teste , resultado_certo in zip(lista_classificada, base_teste["Classe"]):
    
    if resultado_certo == 1 and resultado_teste == 1:
        # mensagens relevantes e que são classificadas como relevantes
        verdadeiros_positivo += 1
        
    elif resultado_certo == 0 and resultado_teste == 1:
        # mensagens irrelevantes e que são classificadas como relevantes
        falsos_positivos += 1
        
    elif resultado_certo == 0 and resultado_teste == 0:
        #mensagens irrelevantes e que são classificadas como irrelevantes
        verdadeiros_negativos += 1
        
    elif resultado_certo == 1 and resultado_teste == 0:
        # mensagens relevantes e que são classificadas como irrelevantes
        falsos_negativos += 1
    
Porcentagem_verdadeiros_positivo = (verdadeiros_positivo / 200) *100

Porcentagem_falsos_positivos = (falsos_positivos / 200) *100

Porcentagem_verdadeiros_negativos = (verdadeiros_negativos / 200) *100

Porcentagem_falsos_negativos = (falsos_negativos / 200) *100

print("Porcentagem de verdadeiros positivos: {}%".format(Porcentagem_verdadeiros_positivo))
print("Porcentagem de falsos positivos: {}%".format(Porcentagem_falsos_positivos))
print("Porcentagem de verdadeiros negativos: {}%".format(Porcentagem_verdadeiros_negativos))
print("Porcentagem de falsos negativos: {}%".format(Porcentagem_falsos_negativos))

Porcentagem de verdadeiros positivos: 55.50000000000001%
Porcentagem de falsos positivos: 1.5%
Porcentagem de verdadeiros negativos: 0.5%
Porcentagem de falsos negativos: 42.5%


### Etapa 5 - Conclusão

O classificador foi construído em função de uma suavização de LaPlace, que consiste em separar as palavras de uma frase e calcular sua probabilidade de acordo com quantas vezes aparece, porém como existem palavras que não apareceram na nossa base, para que sua probabilidade não seja zero, adicionamos 1 para o divisor, assim permitindo que uma conta seja feita. Analisando um pouco as frases de dupla negação, o que acontece no modelo, é que como essas palavras são analisadas separadamente, é como se tivesse um acrescimo em palavras negativas, em vez de uma frase positiva. Porém, não faz muita diferença para o modelo atual, já que as categorias estão divididas em "relevante" e "irrelevante", agora, se fossem divididas em "Bom" e "Ruim" seria outra história.

Algumas porcentagens importantes para análise:

- Verdadeiros positivos: 55.5%
- Falsos positivos: 1.5%
- Verdadeiros negativos: 0.5%
- Falsos negativos: 42.5%

In [14]:
recall = verdadeiros_positivo / (verdadeiros_positivo + falsos_negativos)
precision = verdadeiros_positivo / (verdadeiros_positivo + falsos_positivos)

print(precision, recall)

0.9736842105263158 0.5663265306122449


Podemos definir a precisão de um modelo de Machine Learning como a proporção de predições corretas de uma categoria em relação a todas as previsões feitas dessa categoria. E temos uma precisão de 0,97.

A medida de recall de um modelo de Machine Learning é definido como a proporção de previsões corretas da categoria alvo, Verdadeiros Positivos em relação a soma dos verdadeiros positivos com os Falsos Negativos. E temos uma recall de 0.56

In [15]:
F_score = 2 * ((precision*recall)/(precision+recall))
F_score

0.7161290322580645

O valor mais alto possível de uma F-score é 1.0, indicando precisão e recuperação perfeitas, e o valor mais baixo possível é 0. Sendo assim temos um F_score de 0.716, indicando que é um modelo de predição bom, mas que aceita melhorias.

Além da suavização de LaPlace, houve uma correção e separação entre os emojis, para que os mesmos não interferissem nas palavras, mas é sempre possivel melhorar o modelo, algumas formas que tenho pesquisado:
- Usando "n-gramas", que basicamente consiste em uma sequência contígua de n itens de uma determinada amostra de texto. esta técnica ajudaria em situações com frases e piadas comuns. Links sobre : https://sebastianraschka.com/Articles/2014_naive_bayes_1.html#n-grams, https://blog.xrds.acm.org/2017/10/introduction-n-grams-need/
- Fazendo um estudo a fundo em o que os emojis significam em certas combinações de frase/palavras, pois assim daria para separar emojis que façam uma diferença significativa no tweet, ou se é apenas um complemento do que está sendo dito. Link sobre: http://www.scielo.br/scielo.php?script=sci_arttext&pid=S0103-18132016000200379
- Assim, para implementar essas duas técnicas acima, seria necessário uma base de tweets maior e com mais categorias, por exemplo, "muito relevante", "relevante", "neutro", "irrelevante" e "muito irrelevante"

Após trabalhar novamente com o classificador, uma ideia que vem na mente, usando os tweets, é a calcular a probabilidade a personalidade da pessoa de acordo com os seus tweets, se é uma pessoa mais tímida ou extrvertida, etc. Outra ideia, mas agora usando diferentes tipos de dado, poderia ser calculado a probabilidade de um animal ser macho ou fêmea, de acordo com seu peso, largura, e etc (sendo diferente para cada espécie).